<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-modules-and-load-functions" data-toc-modified-id="Import-modules-and-load-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import modules and load functions</a></span></li><li><span><a href="#Load-misc.-functions" data-toc-modified-id="Load-misc.-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load misc. functions</a></span></li><li><span><a href="#Get-repository-and-dataverse-info" data-toc-modified-id="Get-repository-and-dataverse-info-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get repository and dataverse info</a></span><ul class="toc-item"><li><span><a href="#Get-IDs-of-any-sub-dataverses-in-the-given-dataverse" data-toc-modified-id="Get-IDs-of-any-sub-dataverses-in-the-given-dataverse-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Get IDs of any sub-dataverses in the given dataverse</a></span></li></ul></li><li><span><a href="#Get-updated-dataset-info" data-toc-modified-id="Get-updated-dataset-info-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get updated dataset info</a></span><ul class="toc-item"><li><span><a href="#Get-dataset-IDs-and-(sub)dataverse-names" data-toc-modified-id="Get-dataset-IDs-and-(sub)dataverse-names-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Get dataset IDs and (sub)dataverse names</a></span></li><li><span><a href="#Get-updated-info-about-dataverse's-dataset-and-files" data-toc-modified-id="Get-updated-info-about-dataverse's-dataset-and-files-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Get updated info about dataverse's dataset and files</a></span></li><li><span><a href="#Join-datasetPIDs-and-datasetInfo-dataframes" data-toc-modified-id="Join-datasetPIDs-and-datasetInfo-dataframes-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Join datasetPIDs and datasetInfo dataframes</a></span></li><li><span><a href="#Export-report-to-CSV" data-toc-modified-id="Export-report-to-CSV-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Export report to CSV</a></span></li></ul></li><li><span><a href="#Get-existing-dataset-info-of-dataverse" data-toc-modified-id="Get-existing-dataset-info-of-dataverse-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Get existing dataset info of dataverse</a></span></li><li><span><a href="#Create-summary-stats-of-dataverse" data-toc-modified-id="Create-summary-stats-of-dataverse-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Create summary stats of dataverse</a></span></li></ul></div>

## Import modules and load functions

In [ ]:
from datetime import datetime, timezone
from functools import reduce
import json
import numpy as np
import pandas as pd
import requests
import sys
import time
from urllib.parse import urlparse


## Load misc. functions

In [ ]:
def list_to_string(list):
    # Alphabetize list in case-insensitive way
    list = sorted(list, key=lambda s: s.casefold())

    # Change list to comma-separated string
    delimiter = ","
    string = delimiter.join(list)
    return string


def string_to_list(string): 
    li = list(string.split(",")) 
    return li


def string_to_datetime(string):
    dateTime = datetime.strptime(string, '%Y-%m-%dT%H:%M:%S%z')
    return dateTime


def make_percent(number):
    return(str(round(number * 100, 2)) + '%')


currentTimeUTC = datetime.now(timezone.utc)
currentTimeLocal = time.strftime('%Y.%m.%d_%H.%M.%S')

## Get repository and Dataverse collection info

In [ ]:
# Get dataverse server and alias from user.
mainDataverseUrl = 'https://dataverse.harvard.edu/dataverse/ifpri'

parsed = urlparse(mainDataverseUrl)
server = parsed.scheme + '://' + parsed.netloc
try:
    mainDataverseAlias = parsed.path.split('/')[2]
except IndexError:
    mainDataverseAlias = ''


In [ ]:
def get_repository_metadatablocks(server):
    repositoryMetadataBlocksApi = '%s/api/v1/metadatablocks' % (server)
    response = requests.get(repositoryMetadataBlocksApi)
    repositoryMetadataBlocks = response.json()

    repositoryMetadataBlockNames = []
    for repositoryMetadataBlock in repositoryMetadataBlocks['data']:
        repositoryMetadataBlockNames.append(repositoryMetadataBlock['name'])
    return repositoryMetadataBlockNames

repositoryMetadataBlockNames = get_repository_metadatablocks(server)


In [ ]:
# Get metadata about dataverse
def get_main_dataverse_json(mainDataverseUrl):
    dataverseInfoApi = '%s/api/dataverses/%s' % (server, mainDataverseAlias)
    response = requests.get(dataverseInfoApi)
    dataverseMetadata = response.json()
    return dataverseMetadata


dataverseMetadata = get_main_dataverse_json(mainDataverseUrl)


In [ ]:
if dataverseMetadata['status'] == 'ERROR' or mainDataverseAlias == '' or dataverseMetadata['data']['id'] == 1:
    print('No dataverse found or dataverse is the root dataverse.\
 Please make sure the dataverse is published and not the repository\'s root dataverse.')

if dataverseMetadata['status'] == 'OK':
    def dataverse_description_exists():
        if 'description' in dataverseMetadata['data']:
            dataverseDescriptionExists = True
        else:
            dataverseDescriptionExists = False
        return dataverseDescriptionExists


    def dataverse_tagline_exists():
        if 'theme' in dataverseMetadata['data'] and 'tagline' in dataverseMetadata['data']['theme']:
            taglineExists = True
        else:
            taglineExists = False
        return taglineExists


    def dataverse_facets():
        dataverseFacetsApi = '%s/api/dataverses/%s/facets' % (server, mainDataverseAlias)
        response = requests.get(dataverseFacetsApi)
        dataverseFacets = response.json()
        facets = []
        for facet in dataverseFacets['data']:
            facets.append(facet)
        return facets


    def dataverse_metadatablocks():
        dataverseMetadatablocksApi = '%s/api/dataverses/%s/metadatablocks' % (server, mainDataverseAlias)
        response = requests.get(dataverseMetadatablocksApi)
        dataverseMetadatablocks = response.json()
        dataverseMetadatablocksList = []
        for dataverseMetadatablock in dataverseMetadatablocks['data']:
            dataverseMetadatablocksList.append(dataverseMetadatablock['name'])
        return dataverseMetadatablocksList


### Get IDs of any Dataverse collections in the given Dataverse collection

In [ ]:
def dataverse_ids():
    mainDataverseID = dataverseMetadata['data']['id']
    dataverseIDs = [mainDataverseID]
    for dataverseID in dataverseIDs:

        getContentsApi = '%s/api/dataverses/%s/contents' % (server, dataverseID)

        response = requests.get(getContentsApi)
        dataverseContents = response.json()

        for i in dataverseContents['data']:
            if i['type'] == 'dataverse':
                dataverseID = i['id']
                dataverseIDs.extend([dataverseID])
    return dataverseIDs

print('\nFound 1 dataverse and %s subdataverses' % (len(dataverse_ids()) - 1))

## Get updated dataset info

### Get dataset IDs and (sub)dataverse names

In [ ]:
# Return dataframe with PIDs of all published datasets and their owning (sub)dataverse
def get_datasetPIDs():
    datasetPIDs = []
    datasetInfoDict = []
    for dataverseID in dataverse_ids():
        getDataverseInfoApi = '%s/api/dataverses/%s' % (server, dataverseID)
        response = requests.get(getDataverseInfoApi)
        dataverseInfo = response.json()
        dataverseName = dataverseInfo['data']['name']
        dataverseAlias = dataverseInfo['data']['alias']

        getDataverseContentsApi = '%s/api/dataverses/%s/contents' % (server, dataverseID)
        response = requests.get(getDataverseContentsApi)
        dataverseContents = response.json()
        for item in dataverseContents['data']:
            if item['type'] == 'dataset':
                datasetPID = item['persistentUrl'].replace('https://doi.org/', 'doi:')
                datasetPIDs.append(datasetPID)

                newRow = {'datasetPID': datasetPID,
                      'dataverseName': dataverseName,
                      'dataverseUrl': '%s/dataverse/%s' % (server, dataverseAlias)
                     }
                datasetInfoDict.append(dict(newRow))
    datasetDataverseInfoDF = pd.DataFrame(datasetInfoDict)
    return datasetDataverseInfoDF


In [ ]:
datasetPIDsCount = len(get_datasetPIDs().index)
datasetPIDsCount

### Get updated info about Dataverse collection's datasets and files

In [ ]:
# Getting this info can be slow. For example, getting the info of ~375 datasets might take about 45 min

# Create list of file types that Dataverse can convert to .tab files during ingest
ingestableFileTypes = ['application/x-rlang-transport', 'application/x-stata-13', 'application/x-spss-por',
                      'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'text/csv', 'text/tsv',
                      'application/x-spss-sav', 'text/comma-separated-values', 'application/x-stata',
                      'application/x-stata-14']

datasetInfoDict = []
datasetCount = 0
unretrievableMetadata = []
for datasetPID in get_datasetPIDs()['datasetPID']:
    getAllVersionsApi = '%s/api/datasets/:persistentId/versions?persistentId=%s' % (server, datasetPID)
    response = requests.get(getAllVersionsApi)
    datasetVersions = response.json()
    
#     print('Getting info about %s' % (datasetPID), end='\r', flush=True)
    
    # Get only datasets with metadata (exclude responses with no values in 'data' key, e.g. deaccessioned datasets)
    if datasetVersions['status'] == 'OK' and len(datasetVersions['data']) > 0:
        
        # Get metadata of latest version
        latestDatasetVersion = datasetVersions['data'][0]
        
        # Get index location of first dataset version
        firstVersion = len(datasetVersions['data']) - 1

        publicationDate = string_to_datetime(datasetVersions['data'][firstVersion]['releaseTime'])
        latestReleaseDate = string_to_datetime(latestDatasetVersion['releaseTime'])
        
        # Get age of dataset from today's date
        delta = currentTimeUTC - publicationDate
        ageOfDataset = delta.days
        
        # Get number of days since last update
        delta = currentTimeUTC - latestReleaseDate
        ageOfLastUpdate = delta.days
        if ageOfLastUpdate < 0:
            ageOfLastUpdate = 0
        
        # Get length of description text
        descriptionLength = 0
        
        for field in latestDatasetVersion['metadataBlocks']['citation']['fields']:
            if field['typeName'] == 'dsDescription':
                # "N/A" is the value assigned when there was no description given (pre Dataverse 4)
                if len(field['value']) == 1 and field['value'][0]['dsDescriptionValue']['value'] == 'N/A':
                    descriptionLength = 0
                else:
                    for i in field['value']:
                        descriptionLength = descriptionLength + len(i['dsDescriptionValue']['value'])

        # See whether CC0, Terms of Use, or Terms of Access metadata exists
        license = latestDatasetVersion.get('license', 'None')

        if 'termsOfUse' in latestDatasetVersion:
            termsOfUse = True
        else:
            termsOfUse = False
            
        if 'termsOfAccess' in latestDatasetVersion:
            termsOfAccess = True
        else:
            termsOfAccess = False

        if license != 'CC0' and termsOfUse == False and termsOfAccess == False:
            termsExist = False
        else:
            termsExist = True

        # Get info about related publication metadata
        relPubCount = 0
        relPubPIDCount = 0
        for field in latestDatasetVersion['metadataBlocks']['citation']['fields']:
            if field['typeName'] == 'publication':
                for value in field['value']:
                    relPubCount += 1
                    if 'publicationIDType' and 'publicationIDNumber' in value:
                        relPubPIDCount += 1
        
        # Show metadatablocks whose fields are used by the dataset
        usedMetadataBlocks = []
        for repositoryMetadataBlockName in repositoryMetadataBlockNames:
            try:
                fieldCount = len(latestDatasetVersion['metadataBlocks'][repositoryMetadataBlockName]['fields'])
                if fieldCount > 0:
                    usedMetadataBlocks.append(repositoryMetadataBlockName)
            except KeyError:
                usedMetadataBlocks = usedMetadataBlocks
        if len(usedMetadataBlocks) == 0:
            usedMetadataBlocks = ''
        else:
            usedMetadataBlocks = list_to_string(usedMetadataBlocks)
        
        # Get number of files
        numberOfFiles = len(latestDatasetVersion['files'])

        # Get file info
        noFileDescriptionCount = 0
        contentTypes = []
        ingestedTabFilesCount = 0
        uningestedTabFilesCount = 0
        restrictedFilesCount = 0
        fileTags = []
        for file in latestDatasetVersion['files']:            
            if 'description' in file:
                noFileDescriptionCount = noFileDescriptionCount
            else:
                noFileDescriptionCount += 1

            contentTypes.append(file['dataFile']['contentType'])

            if file['restricted'] == True:
                restrictedFilesCount += 1

            contentType = file['dataFile']['contentType']
            fileSize = file['dataFile']['filesize']
            fileName = file['dataFile']['filename'].lower()
            if contentType in ingestableFileTypes:
                # If it's an RData file that's 1MB or smaller, count it as an uningested file that may be ingested
                if contentType == 'application/x-rlang-transport' and fileSize < 1000001:
                    uningestedTabFilesCount += 1
                # If it's another ingestable file type that's 150MB or smaller, count it as an uningested file that may be ingested
                elif fileSize < 150000001:
                    uningestedTabFilesCount += 1
            # For some CSV files, Harvard Dataverse repository records a generic contentType, so we need to look at the file name extension instead
            if '.csv' in fileName and fileSize < 150000001:
                uningestedTabFilesCount += 1
            if contentType == 'text/tab-separated-values':
                ingestedTabFilesCount += 1

            try:
                for tags in file['categories']:
                    fileTags.append(tags)
            except KeyError:
                fileTags = fileTags

        ingestableTabularFileCount = uningestedTabFilesCount + ingestedTabFilesCount

        if len(fileTags) == 0:
            fileTagsExist = False
        else:
            fileTagsExist = True

        if len(contentTypes) == 0:
            uniqueContentTypes = 'NA'
        else:
            uniqueContentTypes = list_to_string(list(set(contentTypes)))

        # Create dictionary
        newRow = {'datasetPID': datasetPID,
                  'datasetPIDUrl' : datasetPID.replace('doi:', 'https://doi.org/'),
                  'numberOfVersions': len(datasetVersions['data']),
                  'numberOfMajorVersions': latestDatasetVersion['versionNumber'],
                  'publicationDate': publicationDate,
                  'latestReleaseDate': latestReleaseDate,
                  'ageOfDataset(Days)': ageOfDataset,
                  'ageOfLastUpdate(Days)': ageOfLastUpdate,
                  'descriptionLenth': descriptionLength,
                  'termsExist': termsExist,
                  'license': license,
                  'termsOfUseExists': termsOfUse,
                  'termsOfAccessExists': termsOfAccess,
                  'relPubCount': relPubCount,
                  'relPubPIDCount': relPubPIDCount,
                  'usedMetadataBlocks': usedMetadataBlocks,
                  'numberOfFiles': numberOfFiles,
                  'noFileDescriptionCount': noFileDescriptionCount,
                  'fileTagsExist': fileTagsExist,
                  'uniqueContentTypes': uniqueContentTypes,
                  'ingestableTabularFileCount': ingestableTabularFileCount,
                  'ingestedTabFilesCount': ingestedTabFilesCount,
                  'uningestedTabFilesCount': uningestedTabFilesCount,
                  'restrictedFilesCount': restrictedFilesCount
                 }
        datasetInfoDict.append(dict(newRow))
        datasetCount += 1
        print('%s of %s (%s)' % (datasetCount, datasetPIDsCount, datasetPID), end='\r', flush=True)
    else:
        unretrievableMetadata.append(datasetPID)
print('Retrieved dataset info: %s' %(datasetCount))
if unretrievableMetadata:
    print('Unretrievable dataset info: %s' % (len(unretrievableMetadata)))
    print(unretrievableMetadata)


In [ ]:
datasetInfoDF = pd.DataFrame(datasetInfoDict)
datasetInfoDF.head(5)


### Join datasetPIDs and datasetInfo dataframes

In [ ]:
dataframes = [get_datasetPIDs(), datasetInfoDF]

# For each dataframe, set the indexes (or the common columns across the dataframes to join on)
for dataframe in dataframes:
    dataframe.set_index(['datasetPID'], inplace=True)

# Merge both dataframes and save new dataframe to "report" variable
report = reduce(lambda left, right: left.join(right, how='outer'), dataframes)

# Remove any rows with missing info (likely due to published datasets where all versions are deaccessioned)
report = report.dropna()

# Reset index
report.reset_index(drop=False, inplace=True)


In [ ]:
report.head(5)

### Export report to CSV

In [ ]:
# Export report to CSV
file = '%s_%s.csv' % (mainDataverseAlias, currentTimeLocal)
report.to_csv(file, index=False)


## Create summary report of Dataverse collection

In [ ]:
# Get count of datasets in the report
datasetCount = len(report.index)


# Get list of metadatablocks used by all datasets
def get_allUsedMetadataBlocks(report):
    allUsedMetadataBlocks = []
    for i in report['usedMetadataBlocks']:
        allUsedMetadataBlocks.extend(list(i.split(",")))

    # Deduplicate, alphabetize and change list to string
    allUsedMetadataBlocks = list_to_string(list(set(allUsedMetadataBlocks)))
    return allUsedMetadataBlocks


# Get list of uniqueContentTypes used by all datasets
def get_allContentTypes(report):
    allContentTypes = []
    for i in report['uniqueContentTypes']:
        if i != 'NA':
            allContentTypes.extend(list(i.split(",")))
    return allContentTypes


In [ ]:
# Create summary
summaryDict = {
    'Summary': mainDataverseAlias,
    'Has description': dataverse_description_exists(),
    'Has tagline': dataverse_tagline_exists(),
    'Number of search facets': len(dataverse_facets()),
    'Metadatablocks enabled': len(dataverse_metadatablocks()) - 1,
    'Dataset count': datasetCount,
    'Versions (avg # of major and minor versions)': round(report['numberOfVersions'].mean(), 2),
    'Major versions (average #)': round(report['numberOfMajorVersions'].mean(), 2),
    'Age of datasets (average)': round(report['ageOfDataset(Days)'].mean(), 2),
    'Dataset description length (avg # of characters)': round(report['descriptionLenth'].mean(), 2),
    'CC0 datasets (% of total datasets)': make_percent(len(report[(report['license'] == 'CC0')]) / datasetCount),
    'No terms (% of datasets with no terms metadata)': make_percent(((~report['termsExist']).values.sum()) / datasetCount),
    'Related pub metadata (% of datasets with rel pub metadata)': make_percent(len(report[(report['relPubCount'] != 0)]) / datasetCount),
    'Related pub PIDs (% of datasets with rel pub PIDs)': make_percent(len(report[(report['relPubPIDCount'] != 0)]) / datasetCount),
    'Metadatablocks used (list)': get_allUsedMetadataBlocks(report),
    'No files (# of datasets with no files)': len(report[(report['numberOfFiles'] == 0)]),
    'File descriptions (% of datasets with 1 or more file descriptions)': make_percent(len(report[(report['noFileDescriptionCount'] != 0)]) / datasetCount),
    'File tags (% of datasets with 1 or more file tags)': make_percent(((report['fileTagsExist']).values.sum()) / datasetCount),
    'Unique file types (count)': len(set(get_allContentTypes(report))),
    'Ingestable tabular data (% of datasets with data files that can be ingested)': make_percent(((len(report[(report['ingestedTabFilesCount'] != 0)])) + (len(report[(report['uningestedTabFilesCount'] != 0)]))) / len(report[(report['numberOfFiles'] != 0)])),
    'Tabular data ingest successes (% of datasets with tabular data that have been ingested)': make_percent(len(report[(report['ingestedTabFilesCount'] != 0)]) / ((len(report[(report['ingestedTabFilesCount'] != 0)])) + (len(report[(report['uningestedTabFilesCount'] != 0)])))),
    'Public files (% of unrestricted files)': make_percent(((report['numberOfFiles'].sum() - report['restrictedFilesCount'].sum()) / report['numberOfFiles'].sum()))
}


In [ ]:
summaryDF = pd.DataFrame.from_records([summaryDict])
summaryDF = summaryDF.set_index('Summary').transpose()
summaryDF

In [ ]:
# Export summary to CSV
file = '%s_summary_%s.csv' % (mainDataverseAlias, currentTimeLocal)
summaryDF.to_csv(file, index=True)